In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
function = kraft.get_ic

In [ ]:
def simulate(shape, dataframe_n_category=None):

    if dataframe_n_category is None:

        vector = np.random.normal(size=shape[1])

        matrix = np.random.normal(size=shape)

    else:

        vector = np.random.randint(0, high=dataframe_n_category, size=shape[1])

        matrix = np.random.randint(0, high=dataframe_n_category, size=shape)

    columns = pd.Index(("Column {}".format(i) for i in range(shape[1])), name="Column")

    return (
        pd.Series(vector, name="Series", index=columns),
        pd.DataFrame(
            matrix,
            index=pd.Index(("Row {}".format(i) for i in range(shape[0])), name="Row"),
            columns=columns,
        ),
    )

## function_heat_map

In [ ]:
shape = (16, 32)

series, dataframe = simulate(shape)

# dataframe.iloc[0] = np.nan

dataframe.iloc[:, 0] = np.nan

dataframe.iloc[-2:, -2:] = np.nan

kraft.function_heat_map(series, dataframe, function)

In [ ]:
shape = (16, 8)

dataframe = simulate(shape)[1]

for tuple_ in (
    (1, 2, 2, 2, 2, 0, 0, 0),
    (1, 1, 2, 2, 2, 0, 0, 0),
):

    series = pd.Series(tuple_, index=dataframe.columns)

    for series_ascending in (None, True, False):

        series.name = "series_ascending={}".format(series_ascending)

        kraft.function_heat_map(
            series,
            dataframe,
            function,
            series_ascending=series_ascending,
            series_data_type="categorical",
        )

In [ ]:
series = pd.read_csv(
    "data/function_heat_map_series.tsv", sep="\t", index_col=0, squeeze=True
)

dataframe = pd.read_csv("data/function_heat_map_dataframe.tsv", sep="\t", index_col=0)

In [ ]:
kraft.function_heat_map(
    dataframe.iloc[0],
    dataframe.iloc[:1],
    function,
    n_sampling=0,
    n_permutation=0,
    plot_std=3,
)

kraft.function_heat_map(
    dataframe.iloc[0],
    dataframe.iloc[:2],
    function,
    n_sampling=0,
    n_permutation=0,
    plot_std=3,
)

In [ ]:
kraft.function_heat_map(
    series,
    dataframe,
    function,
    n_job=4,
    n_sampling=10,
    n_permutation=10,
    series_data_type="binary",
)

In [ ]:
for shape in (
    (1, 8),
    (2, 8),
    (8, 8),
    (64, 8),
    (8, 2),
    (8, 4),
    (8, 256),
):

    series, dataframe = simulate(shape)

    kraft.function_heat_map(series, dataframe, function, n_extreme=None)

## function_heat_map_summary

In [ ]:
n_column = 16

series, dataframe = simulate((16, n_column))

dataframe_3 = simulate((8, n_column), 4)[1]

dataframe_2 = simulate((4, n_column), 2)[1]

dataframe.drop(dataframe.columns[-3], axis=1, inplace=True)

dataframe_3.drop(dataframe_3.columns[-2], axis=1, inplace=True)

dataframe_2.drop(dataframe_2.columns[-1], axis=1, inplace=True)

dataframe_dicts = {
    "Continuous": {"dataframe": dataframe, "data_type": "continuous"},
    "Categorical": {"dataframe": dataframe_3, "data_type": "categorical"},
    "Binary": {"dataframe": dataframe_2, "data_type": "binary"},
}

scores_dicts = {
    name: kraft.function_heat_map(
        series, dataframe_dict["dataframe"], function, plot=False
    )
    for name, dataframe_dict in dataframe_dicts.items()
}

for plot_only_shared in (False, True):

    kraft.function_heat_map_summary(
        series, dataframe_dicts, scores_dicts, plot_only_shared=plot_only_shared
    )